In [ ]:
import os
from src import RASPRoutines

RASP = RASPRoutines.RASP_Routines()

In [ ]:
""" the command below will re-run colocalisation analysis on alraedy analsed data

analysis_file: an analysis file .csv

threshold: a photon threshold

protein_string: a string for images with protein in

cell_string: a string for cellular images

"""

analysis_file = os.path.abspath(
    r"example_images_analysis/spot_analysis.csv"
)  # example file, replace as you wish. Always input as the absolute path of a raw string
threshold = 500.
protein_string = "C1"
cell_string = "C0"

cell_punctum_analysis_AT, cell_punctum_analysis_UT = (
    RASP.count_puncta_in_individual_cells_threshold(
        analysis_file, threshold, protein_string=protein_string, cell_string=cell_string
    )
)